## Load packages

In [2]:
using SnpArrays, JLD, CSV, DataFrames

## Load data

In [3]:
df = CSV.read("data/pheno/pheno.csv", DataFrame);

In [4]:
# Load covariates
X  = Matrix{Float32}(df[!,["Age_cont","Sex"]])
X  = (X .- mean(X,dims=1))./std(X,dims=1);

In [5]:
# Load and project SNPs
data = SnpData("data/plink/geno-ld08")
S    = zeros(Float32,size(data.snparray)...)
Base.copyto!(S,data.snparray, model=ADDITIVE_MODEL, impute=true, center=true, scale=true)
S  .-= X*((X'X)\(X'S));

## Compute kinship

In [6]:
# Full kinship matrix
K = S*S' ./ (2*size(S,2))
JLD.save("data/kinship/kinship_projected.jld","K",K)

In [ ]:
# LOCO kinship matrices
for i in 1:25
    println("Computing LOCO projected kinship for chr $i")
    cols = findall(data.snp_info.chromosome .!= string(i))
    S_ = @view S[:,cols]
    K = S_*S_' ./ (2*size(S_,2))
    JLD.save("data/kinship/kinship_projected_loco_$i.jld","K",K)
end